Code to scrape websites and return json objects.

Called OPaL: Opposing Positions and Lingo

In [8]:
from url_catcher import get_all_news_urls, main

ModuleNotFoundError: No module named 'url_catcher'

In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time

In [ ]:
def make_request(urls:list):
    responses = []
    for url in urls:
        try:
            response = requests.get(url, timeout=5)
            response.raise_for_status()
            responses.append(response.text)
        except requests.exceptions.RequestException as exception:
            if hasattr(exception, 'response') and exception.response is not None:
                status_code = exception.response.status_code
                raise requests.HTTPError(f"HTTP Error: {status_code} occured", 
                                    response=exception.response)
            raise requests.HTTPError("Non-Specific Request Error: 500")
    return responses

In [ ]:
# 1819 specific parser

def article_parser_1819(urls):
    #create request object by calling the make_request function
    
    request_responses = make_request(urls)
    all_articles = []

    #parse the request_responses
    for request_response in request_responses:
        soup = BeautifulSoup(request_response, 'html.parser')
        json_soup = {
            'title': '',
            'author': '',
            'date': '',
            'line_count': 0,
            'line_content': {}
            }

        #pull the title element and add it to the json object
        title = soup.title.string
        title = title.strip()
        json_soup['title'] = title

        #extract author and date
        author_date_div = soup.find('div', class_='author-date')
        author_name = author_date_div.find('a').text
        date = author_date_div.text.split('|')[1].strip()

        #assign author and date to json object
        json_soup['author'] = author_name
        json_soup['date'] = date

        #pull the paragraph elements and create an empty array for the contents
        paragraphs = soup.find_all(['p'])
        paragraph_texts = []


        for p in paragraphs:
            # Get the text and strip whitespace
            text = p.get_text().strip()
            # Split by line breaks that might be in the HTML
            lines = text.split('\n')
            # Add each non-empty line
            for line in lines:
                if line.strip():  # Only add non-empty lines
                    paragraph_texts.append(line.strip())
        
        # create a json object of paragraph lines and line contents
        json_soup['line_count'] = len(paragraph_texts)
        for i, line in enumerate(paragraph_texts, 1):
            json_soup['line_content'][f"line {i}"] = line
        
        all_articles.append(json_soup)

    return json.dumps(all_articles, indent=4, ensure_ascii=False)


In [ ]:
if __name__ == "__main__":
    
    # Get URLs from url_catcher
    base_url = 'https://1819news.com/'
    suffix = '/news/item'
    max_pages = 3  # Adjust this number based on how many pages you want to scrape
    
    # Get the URLs using the url_catcher function
    urls = get_all_news_urls(base_url, suffix, max_pages)
    
    # Process those URLs with your article parser
    result = article_parser_1819(urls)
    print(result)

NameError: name 'get_all_news_urls' is not defined